# Default Import

In [1]:
from ris import pysqldb
from IPython.display import Markdown, clear_output
from sqlalchemy import create_engine
import ris
import getpass
import datetime 
import pandas as pd
import numpy as np
import os
import requests

timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')
ts = datetime.datetime.now().strftime('%Y-%m-%d')

Markdown('<strong>Notebook run on: {} | by {} | Using ris library version: {} | File Location: {}'.format(
    timestamp, getpass.getuser(), ris.__version__, os.getcwd()
))


<strong>Notebook run on: 2020-10-08 17:00 | by soge | Using ris library version: 1.5.6 | File Location: E:\RIS\Staff Folders\Samuel\Requests\SH\Crash_Data_View

# Db connections

In [6]:
try: 
    sql.params['user']
except:
    sql = pysqldb.DbConnect(type='MS', server='dotdevgissql01', database='RISCRASHDATA')


Additional database connection details required:
User name (ris):soge
Password (ris)········
Database connection (PG) to ris on dotdevrhpgsql01 - user: soge 
Connection established 2020-03-11 16:17:47


In [7]:
#sip = pysqldb.DbConnect(server='dotpgsql01', database='sip', type='PG', user = db.params['user'], password = db.params['password'])
#forms = pysqldb.DbConnect(server='dot55sql01', database='forms', type='MS', user = 'arcgis', password = 'arcgis')

In [ ]:
query_string = """

* Author: Ranga Tanjore, 03/13/2018
* NONMV = 1 should be excluded
* VOID_STATUS_CD = 'Y' should be excluded
* PCT, YY, SEQ, NODEID, PED_NONPED, OCCURRENCE_DATETIME, ON_STREET, CROSS_STREET, OFF_STREET, INJURED_COUNT, KILLED_COUNT, ID
* All new Crashes after 12/31/2016 --> Years 2017, 2018, etc...
* Changed @nodeslist @masternodeslist varchar(max); this version has the clion 19d masterid  
************************************************************************************************************************************/
SET NOCOUNT ON
DECLARE @NODEID varchar(20)

--declare @nodeslist varchar(max) = '13332,13336,13337,13338,13349,13398,13399,13400,13401,13402,13409,13477,13478,13479,13484,13485,13562,13476,13567,13568,13563,13570,13571,13628,13569,65459,13629,13630,13631,13632,13633';
set @nodeslist = isnull(@nodeslist, '44725,44783,44784,44793,48606,48614,48615,48616,48630,48632,48633,9036912,9037711,9037712,9037719,9037720,9037721,9037722,9037723,9037724,9037731,9037732,9037739,')
SET @masternodeslist = isnull(@masternodeslist, '44784,44793,48606,48616,48630,48632,48633,')
SET @masternodeslist = @masternodeslist + ','
--select * from dbo.fnGetIDs('27267,27272,29117,29119,29122,29124,29127,29129,29130,29131,29132,67620,9004944,9004947,9004954,9004959,9004960,9004965,9004966,9004970,9004973')
--DECLARE @START_BEFORE_DT varchar(20) = '01/01/2012'
--DECLARE @END_BEFORE_DT varchar(20) = '01/01/2015'
--DECLARE @START_AFTER_DT varchar(20) = '02/01/2015'
--DECLARE @END_AFTER_DT varchar(20) = '02/01/2018'

SET @START_BEFORE_DT = isnull(@START_BEFORE_DT, '01/01/2013')
SET @END_BEFORE_DT = isnull(@END_BEFORE_DT, '12/31/2019')
SET @START_AFTER_DT = isnull(@START_AFTER_DT, '01/01/2013')
SET @END_AFTER_DT = isnull(@END_AFTER_DT, '12/31/2019')

DECLARE @tblNodes TABLE(NodeID varchar(20))
DECLARE @N1 INT
DECLARE @DT_BEFORE_3 DATETIME = DATEADD(yy, -1, @END_BEFORE_DT)
DECLARE @DT_BEFORE_2 DATETIME = DATEADD(yy, -2, @END_BEFORE_DT)
DECLARE @DT_BEFORE_1 DATETIME = DATEADD(yy, -3, @END_BEFORE_DT)
DECLARE @DT_AFTER_1 DATETIME = DATEADD(yy, 1, @START_AFTER_DT) 
DECLARE @DT_AFTER_2 DATETIME = DATEADD(yy, 2, @START_AFTER_DT) 
DECLARE @DT_AFTER_3 DATETIME = DATEADD(yy, 3, @START_AFTER_DT) 

DECLARE @tblOut TABLE(
       [REQID] [int] identity(1,1),
       [ANUM_PCT] varchar(10) NOT NULL,
       [ANUM_YY] varchar(2) NOT NULL,
       [ANUM_SEQ] varchar(5) NOT NULL,
       [NODEID] varchar(20) NOT NULL,
       [PED_NONPED] [varchar](5) NULL,
       [OCCURRENCE_DATETIME] varchar(20) NOT NULL,
       [ON_STREET] [varchar](250) NULL,
       [CROSS_STREET] [varchar](250) NULL,
       [OFF_STREET] [varchar](250) NULL,
       [INJURED_COUNT] [int] NULL,
       [KILLED_COUNT] [int] NULL,
       [VIC_NUM] [int] NOT NULL,
       [ID] [varchar](20) NULL,
       [DateYear] [varchar](20) NULL,
       INTEGRATION_ID BIGINT
)

;with ctPrevent
as
(
       select ACCIDENT_ID, 
       max(preventable_ped) as preventable_ped, 
       max(preventable_bike) as preventable_bike, 
       max(preventable_collision) as preventable_collision
       FROM 
       (
              select c.INTEGRATION_ID as ACCIDENT_ID, 
              case when PERSON_ROLE_CODE in ('Pedestrian', 'In-Line Skater') then 1 else 0 end preventable_ped, 
              case when PED_NONPED in ('Bicyclist') then 1 else 0 end preventable_bike,
              0 as preventable_collision 
              from WC_ACCIDENT_VICTIM_F v
              inner join (
                     select c1.* from Forms.dbo.WC_ACCIDENT_F c1
                     inner join (select * from dbo.fnGetIDs(@masternodeslist)) n1 ON (c1.masterid=n1.ID)
                     union
                     select c1.* from Forms.dbo.WC_ACCIDENT_F c1
                     inner join (select * from dbo.fnGetIDs(@nodeslist)) n1 ON (c1.NODEID=n1.ID)
              ) c
              on v.ACCIDENT_ID = c.INTEGRATION_ID
              where inj_killed in ('Injured', 'Killed')
              and (PERSON_ROLE_CODE in ('Pedestrian', 'In-Line Skater') 
              or 
              PED_NONPED in ('Bicyclist'))
              union
              select INTEGRATION_ID, 0, 0, 1 --as preventable_collision 
              from (
                     select c1.* from Forms.dbo.WC_ACCIDENT_F c1
                     inner join (select * from dbo.fnGetIDs(@masternodeslist)) n1 ON (c1.masterid=n1.ID)
                     union
                     select c1.* from Forms.dbo.WC_ACCIDENT_F c1
                     inner join (select * from dbo.fnGetIDs(@nodeslist)) n1 ON (c1.NODEID=n1.ID)
              ) c
              where ACCIDENT_DIAGRAM IN ('LEFT_TURN', 'RIGHT_TURN', 'RIGHT_ANGLE')
       
       ) as tbl
       GROUP BY ACCIDENT_ID
)
--select top 10 * from WC_ACCIDENT_F
--select top 10 * from WC_ACCIDENT_VICTIM_F
--select * from ctPrevent

--INSERT INTO @tblOut
select DISTINCT 'F '+c.SRC_POLICE_PRECINCT as ANUM_PCT, ACCIDENT_DT,
              RIGHT('00000' + SUBSTRING(RIGHT('00000' + convert(varchar(20), CAST(c.INTEGRATION_ID as bigint)),10),6,2),2) as ANUM_YY,
              RIGHT('00000' + SUBSTRING(RIGHT('00000' + convert(varchar(20), CAST(c.INTEGRATION_ID as bigint)),10),1,5),5)  as ANUM_SEQ,
              convert(varchar, isnull(nodeid, MASTERID)) as NODEID,         
              convert(varchar, MASTERID) as MASTERID,         
              isnull(VIC_PED_NONPED,'') as PED_NONPED,
              CONVERT(varchar, DATEADD(minute, CAST((CAST(left(right('0'+convert(varchar, ACCIDENT_TIME_WID),6),2) as INT) *60 + CAST(substring(right(convert(varchar, ACCIDENT_TIME_WID),4),1,2) as int)) as INT), convert(datetime, ACCIDENT_DT)), 100)  as OCCURRENCE_DATETIME,
              YEAR(ACCIDENT_DT) as CASE_YR,
              rtrim(isnull(c.SRC_ON_STREET,'')) as ON_STREET, 
              rtrim(isnull(c.SRC_CROSS_STREET,'')) as CROSS_STREET , 
              rtrim(isnull(SRC_OFF_STREET,'')) as OFF_STREET, 
              VIC_INJURED_COUNT as INJURED_COUNT,
              VIC_KILLED_COUNT as KILLED_COUNT,
              (CASE WHEN VIC_INTEGRATION_ID IS NULL THEN 0 ELSE CAST(RANK() OVER(PARTITION BY c.ACCIDENT_ID ORDER BY VIC_INTEGRATION_ID) as INT) END) as VIC_NUM,
              RIGHT('00000' + convert(varchar(20), CAST(c.INTEGRATION_ID as bigint)),10) as ID,
              (CASE 
                     WHEN (c.ACCIDENT_DT >= @DT_BEFORE_3) AND (c.ACCIDENT_DT <= @END_BEFORE_DT) THEN 'Before 3'
                     WHEN (c.ACCIDENT_DT >= @DT_BEFORE_2) AND (c.ACCIDENT_DT <= @END_BEFORE_DT) THEN 'Before 2'
                     WHEN (c.ACCIDENT_DT >= @DT_BEFORE_1) AND (c.ACCIDENT_DT <= @END_BEFORE_DT) THEN 'Before 1'
                     WHEN (c.ACCIDENT_DT <= @DT_AFTER_1) AND (c.ACCIDENT_DT >= @START_AFTER_DT) THEN 'After 1'
                     WHEN (c.ACCIDENT_DT <= @DT_AFTER_2) AND (c.ACCIDENT_DT >= @START_AFTER_DT) THEN 'After 2'
                     WHEN (c.ACCIDENT_DT <= @DT_AFTER_3) AND (c.ACCIDENT_DT >= @START_AFTER_DT) THEN 'After 3'
                     ELSE 'Out of Range'
                     END) as DateYear,
                     c.INTEGRATION_ID,
                     p.PREVENTABLE_PED,
                     p.PREVENTABLE_BIKE,
                     p.PREVENTABLE_COLLISION,
                     PREVENTABLE_DIRECTION,
                     TRAFFIC_CONTROL,
                     ACCIDENT_DIAGRAM,
                     ACCIDENT_DESC,
                     X = X_COORD, 
                     Y = Y_COORD, 
                     VICTIM_AGE,
                     VICTIM_SEX
from (
              select c.SRC_POLICE_PRECINCT, c.INTEGRATION_ID, c.ACCIDENT_DT, c.ACCIDENT_TIME_WID, c.SRC_ON_STREET, c.SRC_CROSS_STREET, c.SRC_OFF_STREET, c.LION_NODE_NUMBER, c.SEGMENT_ID , 
              c.nodeid, 
              c.MASTERID,
              v.VICTIM_NUM,
              v.INTEGRATION_ID as VIC_INTEGRATION_ID,
              v.PERSON_ROLE_CODE as VIC_PERSON_ROLE_CODE,  
              v.ACCIDENT_ID,
              /** Bug found by Hisa for In-Line skater Exception 4/24/2020 
              (CASE v.PED_NONPED WHEN 'Bicyclist' THEN 'B' WHEN 'Occupant' THEN 'N' WHEN 'Pedestrian' THEN 'P' ELSE PED_NONPED END) as VIC_PED_NONPED,
              **/
              (CASE WHEN v.PED_NONPED = 'Bicyclist' THEN 'B' 
                     WHEN v.PED_NONPED = 'Occupant' AND v.PERSON_ROLE_CODE <>'In-Line Skater' THEN 'N' 
                     WHEN (v.PED_NONPED = 'Pedestrian' OR v.PERSON_ROLE_CODE ='In-Line Skater') THEN 'P' 
                     ELSE PED_NONPED END) as VIC_PED_NONPED,
              (CASE v.INJ_KILLED  WHEN 'Injured' THEN 1 ELSE 0 END) as VIC_INJURED_COUNT,
              (CASE v.INJ_KILLED  WHEN 'Killed' THEN 1 ELSE 0 END) as VIC_KILLED_COUNT,
              1 as SEL,
              CAST(c.X_COORD as INT) as X_COORD, 
              CAST(c.Y_COORD as INT) as Y_COORD, 
              --c.SRC_ON_STREET, 
              --c.SRC_CROSS_STREET, 
              c.TRAFFIC_CONTROL,
              v.VICTIM_AGE,
              v.VICTIM_SEX,
              ACCIDENT_DIAGRAM,
              ACCIDENT_DESC,
              dbo.fnGetPreventableDirection(c.INTEGRATION_ID) as PREVENTABLE_DIRECTION
              from (
                     select c1.* from Forms.dbo.WC_ACCIDENT_F c1
                     inner join (select * from dbo.fnGetIDs(@masternodeslist)) n1 ON (c1.masterid=n1.ID)
                     union
                     select c1.* from Forms.dbo.WC_ACCIDENT_F c1
                     inner join (select * from dbo.fnGetIDs(@nodeslist)) n1 ON (c1.NODEID=n1.ID)
              ) c
              left outer join (
                     select a.* from Forms.dbo.WC_ACCIDENT_VICTIM_F a where INJ_KILLED in ('Injured', 'Killed')  /* -- as per Seth don't group by age; meeting 10/19/2018*/
                     --and PERSON_ROLE_CODE in ('Driver' , 'In-Line Skater', 'Other', 'Passenger', 'Pedestrian')               
              )as v on c.INTEGRATION_ID = v.ACCIDENT_ID 
              WHERE ((c.ACCIDENT_DT between @START_BEFORE_DT and @END_BEFORE_DT) OR (c.ACCIDENT_DT between @START_AFTER_DT and @END_AFTER_DT))
              AND c.ACCIDENT_DT>'12/31/2016'
              AND isnull(c.NONMV,0)<>1 
              AND Isnull(c.VOID_STATUS_CD,'')<>'Y'
) as c 
left outer join ctPrevent p on (c.INTEGRATION_ID=p.ACCIDENT_ID)
order by ACCIDENT_DT ASC, VIC_NUM
"""